In [ ]:
!unzip /content/drive/MyDrive/en_US.twitter.txt.zip

Archive:  /content/drive/MyDrive/en_US.twitter.txt.zip
  inflating: en_US.twitter.txt       


In [30]:
# load the data
with open("./twitter_sm.txt", "r") as f:
    data = f.read()

# print part of the data
display(data[0:300])


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

In [31]:
# tokenize the data
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [word_tokenize(senctence) for senctence in sent_tokenize(data)]

# display some sentences
print(sentences[:5])

[['How', 'are', 'you', '?'], ['Btw', 'thanks', 'for', 'the', 'RT', '.'], ['You', 'gon', 'na', 'be', 'in', 'DC', 'anytime', 'soon', '?'], ['Love', 'to', 'see', 'you', '.'], ['Been', 'way', ',', 'way', 'too', 'long', '.']]


In [32]:
# save resulted lists 
import pickle

with open("./sentences", "wb") as fp:   #Picklin
  pickle.dump(sentences, fp)

In [33]:
# preprocess the data
## no need to remove stop words
## no need to stem or lemmize the words
## it is optional to remove punctuation but we chose not to do
## it is important to lowercasing the words

for i, sent in enumerate(sentences):
  for j, word in enumerate(sent):
    sentences[i][j] = word.lower()

# display some sentences
print(sentences[:5])

[['how', 'are', 'you', '?'], ['btw', 'thanks', 'for', 'the', 'rt', '.'], ['you', 'gon', 'na', 'be', 'in', 'dc', 'anytime', 'soon', '?'], ['love', 'to', 'see', 'you', '.'], ['been', 'way', ',', 'way', 'too', 'long', '.']]


In [34]:
# save resulted lists 
with open("./lower_sentences", "wb") as fp:   #Picklin
  pickle.dump(sentences, fp)

In [35]:
# calculate unigram and bigram
def n_grams_counter(data, n, start_token='', end_token = ''):
    n_grams = {}

    for sentence in data: 
        sentence = [start_token] * n + sentence + [end_token]
        sentence = tuple(sentence)
        
        if n==1: #unigram
          m = len(sentence)
        else:
          m = len(sentence) - 1

        for i in range(m): 
            n_gram = sentence[i:i+n]
            if n_gram in n_grams.keys(): 
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
    return n_grams

unigram = n_grams_counter(sentences, 1, 'sos', 'eos')
bigram = n_grams_counter(sentences, 2, 'sos', 'eos')

# display some results
print(dict(list(unigram.items())[:5]))
print(dict(list(bigram.items())[:5]))

{('sos',): 56910, ('how',): 1547, ('are',): 3366, ('you',): 12281, ('?',): 8502}
{('sos', 'sos'): 56910, ('sos', 'how'): 391, ('how', 'are'): 83, ('are', 'you'): 490, ('you', '?'): 196}


In [36]:
# save resulted lists 

with open("./unigrams", "wb") as fp:   #Picklin
  pickle.dump(unigram, fp)

with open("./bigrams", "wb") as fp:   #Picklin
  pickle.dump(bigram, fp)


In [118]:
# get the vocab
## load sentences
 
with open("./lower_sentences", "rb") as fp:
  sentences = pickle.load(fp)


vocab = []
words = []
for sent in sentences:
  for word in sent:
    words.append(word)
print(len(words))
vocab = list(set(words))
print(len(vocab))


746976
41726


In [38]:
# save resulted lists 

with open("./vocabulary", "wb") as fp:   #Picklin
  pickle.dump(vocab, fp)

In [47]:
def get_propability_of_specific_word(word, current_token, unigram, bigram, vocabulary_size, k=1.0):
    
    current_token = tuple(current_token)

    current_token_count = unigram[current_token] if current_token in unigram else 0
    
    denominator = current_token_count + k * vocabulary_size

    n_plus1_gram = current_token + (word,)
  
    n_plus1_gram_count = bigram[n_plus1_gram] if n_plus1_gram in bigram else 0
        
    numerator = n_plus1_gram_count + k

    probability = numerator / denominator
    
    return probability


In [141]:
from operator import itemgetter
def get_propability_of_all_words(current_token, unigram, bigram, vocabulary, k=1.0):
   
    current_token = tuple(current_token)
    vocabulary = vocabulary + ["", ""]
    vocabulary_size = len(vocabulary)
    
    probabilities = {}
    for word in vocabulary:
        probability = get_propability_of_specific_word(word, current_token, 
                                           unigram, bigram, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability
    #dict(list(probabilities.items())[:3])
    return dict(sorted(probabilities.items(), key = itemgetter(1), reverse = True)[:10])
    # probabilities = []
    # for word in vocabulary:
    #     probability = get_propability_of_specific_word(word, current_token, unigram, bigram, vocabulary_size, k=k)
    #     probabilities.append(probability)
    # import numpy as np
    # return np.argmax(probabilities)

In [153]:
# # test your code
# sents = [['i', 'like', 'a', 'cat'],
#              ['this', 'dog', 'is', 'like', 'a', 'cat']]
# unique_words = list(set(sents[0] + sents[1]))
# unigram_counts = n_grams_counter(sents, 1)
# bigram_counts = n_grams_counter(sents, 2)
# get_propability_of_all_words("this", unigram_counts, bigram_counts, unique_words, k=1)

vocab[:10]

['clothing',
 'gypsy',
 'victims',
 'hopping',
 'antioch',
 'beauts',
 'pastries',
 'mulholland',
 '1682',
 'fiat']

In [150]:
# test
get_propability_of_all_words("u", unigram, bigram, vocab, k=1)

{'can': 0.0010624538063562453,
 'have': 0.000877679231337768,
 '!': 0.000877679231337768,
 'know': 0.0007852919438285292,
 'do': 0.0007621951219512195,
 'r': 0.0006698078344419808,
 'get': 0.0006236141906873614,
 '?': 0.0006236141906873614,
 'got': 0.0005543237250554324,
 'guys': 0.0005543237250554324}

In [159]:
# try with trigram
trigram = n_grams_counter(sentences, 3)

with open("./trigrams", "wb") as fp:   #Picklin
  pickle.dump(trigram, fp)


In [165]:
get_propability_of_all_words(["how", "do"], bigram, trigram, vocab, k=1)

{'you': 0.0008374007082017418,
 'i': 0.000406737486840846,
 'we': 0.00019140587616039813,
 'u': 0.0001435544071202986,
 'evangelicals': 4.785146904009953e-05,
 'travel': 4.785146904009953e-05,
 'vampire': 4.785146904009953e-05,
 'april': 4.785146904009953e-05,
 'trannys': 4.785146904009953e-05,
 'ya': 4.785146904009953e-05}